## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nanortalik,GL,60.1432,-45.2371,19.22,74,100,20.92,light snow
1,1,Ushuaia,AR,-54.8000,-68.3000,44.26,70,40,28.77,scattered clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,50.11,57,75,26.46,broken clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.19,78,14,15.01,light rain
4,4,Rikitea,PF,-23.1203,-134.9692,77.11,71,100,5.77,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_temp_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]


In [15]:
# 4a. Determine if there are any empty rows.
user_temp_df.isnull().sum()


City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_temp_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Punta Arenas,CL,50.11,broken clouds,-53.1500,-70.9167,
3,Atuona,PF,78.19,light rain,-9.8000,-139.0333,
4,Rikitea,PF,77.11,overcast clouds,-23.1203,-134.9692,
5,Lundazi,ZM,62.24,broken clouds,-12.2929,33.1782,
6,Banda Aceh,ID,76.42,broken clouds,5.5577,95.3222,
7,Hermanus,ZA,70.90,overcast clouds,-34.4187,19.2345,
8,Puerto Ayora,EC,74.64,overcast clouds,-0.7393,-90.3518,
9,Sechura,PE,74.41,broken clouds,-5.5569,-80.8222,
10,Saldanha,ZA,68.05,broken clouds,-33.0117,17.9442,
13,Bubaque,GW,77.99,broken clouds,11.2833,-15.8333,


In [26]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    params = {"radius": 5000,
              "types": "lodging",
              "location": f"{lat},{lng}",
              "key": g_key}

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print("Hotel found!")
    except:
        print("Hotel not found... Skipping")
        pass
        

Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel not found... Skipping
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel not found... Skipping
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel fo

In [29]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()
#I think Pass did it for me?

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>County</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>High Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()

marker = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(marker)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))